# A Design Space Study for LISTA and Beyond



---
Copyright 2021 Google Research, Licensed under the Apache License, Version 2.0


## 0. Install package and download checkpoints/dictionary.



In [ ]:
%%capture
#@title
import os
import sys

# Download source code.
if "lista_design_space" not in os.getcwd():
  !git clone --depth 1 https://github.com/google-research/google-research
  os.chdir('google-research/lista_design_space')
  sys.path.append('.')
  !pip install -r requirements.txt
else:
  !git pull

os.mkdir('data')
!wget https://storage.googleapis.com/gresearch/lista_design_space/A_250_500.npy -P data
os.mkdir('checkpoints')
!wget https://storage.googleapis.com/gresearch/lista_design_space/lista.h5 -P checkpoints
!wget https://storage.googleapis.com/gresearch/lista_design_space/lfista.h5 -P checkpoints
!wget https://storage.googleapis.com/gresearch/lista_design_space/dense.h5 -P checkpoints
!wget https://storage.googleapis.com/gresearch/lista_design_space/searched.h5 -P checkpoints

## 1. Generate test data.


In [ ]:
!python data_generator.py

2021-03-31 23:15:45.169813: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


## 2. Define loss function and metrics.

In [ ]:
import os
import sys

import numpy as np
import tensorflow as tf
import lista_models


class MSE(tf.keras.losses.Loss):
  """MSE loss for LISTA."""

  def __init__(self, dict_shape, **kwargs):
    super(MSE, self).__init__(**kwargs)
    self.dict_shape = dict_shape

  def call(self, y_true, y_pred):
    loss = tf.nn.l2_loss(y_pred[:, -self.dict_shape[1]:] - y_true)
    return loss


class NMSE(tf.keras.metrics.Mean):
  """NMSE metrics for LISTA."""

  def __init__(self, name, dict_shape, dtype=None):
    super(NMSE, self).__init__(name=name, dtype=dtype)
    self.dict_shape = dict_shape

  def update_state(self, y_true, y_pred, sample_weight=None):
    mse = tf.reduce_mean(
        tf.square(y_true - y_pred[:, -self.dict_shape[1]:]), axis=-1)
    nmse_denom = tf.reduce_mean(tf.square(y_true), axis=-1)
    nmse = mse / nmse_denom
    nmse_val = 10.0 * tf.math.log(nmse) / tf.math.log(10.0)
    return super(NMSE, self).update_state(nmse_val, sample_weight=sample_weight)

## 3. Define input dataset.

In [ ]:
def input_fn(mode, batch_size, dict_shape):
  """Input dataset for LISTA."""
  filename = 'sc_{}_data.npy'.format(mode)
  data = np.load(
      os.path.join('./data', filename),
      allow_pickle=True).astype(np.float32)
  dataset = tf.data.Dataset.from_tensor_slices(data)

  def _parser(value):
    return value[:dict_shape[0]], value[dict_shape[0]:]

  dataset = dataset.map(_parser)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return dataset

## 4. Define test code.

In [ ]:
_NUM_LAYERS = 16
_LAMBDA = 0.4
_NUM_TEST_IMAGES = 10240
_TEST_BATCH_SIZE = 1024

def test(model_name):
  if model_name == 'lista':
    arch_str = '0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0'
  elif model_name == 'lfista':
    arch_str = '0_0_1_1_1_1_1_1_1_1_1_1_1_1_1_1'
  elif model_name == 'dense':
    arch_str = '0_0_1_3_7_15_31_63_127_255_511_1023_2047_4095_8191_16383'
  elif model_name == 'searched':
    arch_str = '0_0_0_2_1_3_14_42_48_249_284_4_152_328_4888_12193'
  else:
    raise ValueError('Model {} is not supported.'.format(model_name))

  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.TPUStrategy(tpu)

  with strategy.scope():
    dictionary = np.load(
        tf.io.gfile.GFile('./data/A_250_500.npy', 'rb'),
        allow_pickle=True).astype(np.float32)
    test_dataset = input_fn('test', _TEST_BATCH_SIZE, dictionary.shape)
    model = lista_models.Lista(dictionary, _LAMBDA, arch_str, _NUM_LAYERS)
    model.build([None, dictionary.shape[0]])
    model.load_weights('./checkpoints/{}.h5'.format(model_name))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=MSE(dictionary.shape),
        metrics=[NMSE('nmse', dictionary.shape)])
    model.evaluate(
        x=test_dataset,
        steps=_NUM_TEST_IMAGES // _TEST_BATCH_SIZE,
        verbose=2)

## 5. Run test.

In [ ]:
test('lista')

Running on TPU  ['10.38.74.210:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.38.74.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.38.74.210:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


10/10 - 3s - loss: 0.8715 - nmse: -4.2895e+01


In [ ]:
test('lfista')

Running on TPU  ['10.38.74.210:8470']


INFO:tensorflow:Initializing the TPU system: grpc://10.38.74.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.38.74.210:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


10/10 - 3s - loss: 0.1473 - nmse: -4.7348e+01


In [ ]:
test('dense')

Running on TPU  ['10.38.74.210:8470']


INFO:tensorflow:Initializing the TPU system: grpc://10.38.74.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.38.74.210:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


10/10 - 3s - loss: 0.1444 - nmse: -4.8065e+01


In [ ]:
test('searched')

Running on TPU  ['10.38.74.210:8470']


INFO:tensorflow:Initializing the TPU system: grpc://10.38.74.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.38.74.210:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


10/10 - 3s - loss: 0.0439 - nmse: -5.4126e+01
